In [171]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

# Getting CSV

In [172]:
import requests
url = "https://raw.githubusercontent.com/Joffreybvn/real-estate-data-analysis/master/data/clean/belgium_real_estate.csv"
res = requests.get(url, allow_redirects=True)
with open ("belgium_real_estate.csv", "wb") as file:
    file.write(res.content)

# Reading CSV

In [173]:
df1 = pd.read_csv("belgium_real_estate.csv")

In [174]:
df1.head()

,Unnamed: 0,postal_code,city_name,type_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,0,1050,Ixelles,0,340000,6,203,1,0,1,0,95,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
1,1,1050,Ixelles,0,520000,4,200,0,0,0,0,69,2,0,to renovate,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
2,2,1050,Ixelles,0,599000,4,160,1,0,1,1,100,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
3,3,1050,Ixelles,0,599000,3,160,1,0,1,1,130,2,0,good,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
4,4,1050,Ixelles,0,575000,3,171,0,0,0,0,46,2,0,just renovated,4.381571,50.822285,Bruxelles-Capitale,Bruxelles


In [175]:
df1.shape

(40395, 19)

In [177]:
df2 = df1[["price", "city_name", "house_area", "number_of_rooms", "swimming_pool"]]
df2.head()

,price,city_name,house_area,number_of_rooms,swimming_pool
0,340000,Ixelles,203,6,0
1,520000,Ixelles,200,4,0
2,599000,Ixelles,160,4,0
3,599000,Ixelles,160,3,0
4,575000,Ixelles,171,3,0


In [178]:
df2.isnull().sum()

price              0
city_name          0
house_area         0
number_of_rooms    0
swimming_pool      0
dtype: int64

In [179]:
df2["number_of_rooms"].unique()

array([ 6,  4,  3,  1,  2,  5, 12,  8,  7, 11, 10, 15,  9, 18, 16, 13, 14])

In [180]:
df3 = df2.copy()
df3["price_per_sqft"] = df3["price"]/df3["house_area"]
df3.head()

,price,city_name,house_area,number_of_rooms,swimming_pool,price_per_sqft
0,340000,Ixelles,203,6,0,1674.876847
1,520000,Ixelles,200,4,0,2600.000000
2,599000,Ixelles,160,4,0,3743.750000
3,599000,Ixelles,160,3,0,3743.750000
4,575000,Ixelles,171,3,0,3362.573099


In [183]:
len(df3.city_name.unique())

1046

In [187]:
location_stats = df3.groupby("city_name")["city_name"].agg("count").sort_values(ascending=False)
location_stats

city_name
Antwerpen        886
Knokke           750
Oostende         685
Gent             634
Uccle            498
                ... 
Borsbeke           1
Bas-Warneton       1
Mohiville          1
Walhorn            1
Lommersweiler      1
Name: city_name, Length: 1046, dtype: int64

In [192]:
len(location_stats[location_stats<=10])

375

In [193]:
location_less_than_10 = location_stats[location_stats<=10]
location_less_than_10

city_name
Baudour          10
Ligne            10
Hertain          10
Melen            10
Torgny           10
                 ..
Borsbeke          1
Bas-Warneton      1
Mohiville         1
Walhorn           1
Lommersweiler     1
Name: city_name, Length: 375, dtype: int64

In [194]:
df3.city_name = df3.city_name.apply(lambda x: "other" if x in location_less_than_10 else x)
len(df3.city_name.unique())

672

In [195]:
df3[df3.house_area/df3.number_of_rooms<300].head()

,price,city_name,house_area,number_of_rooms,swimming_pool,price_per_sqft
0,340000,Ixelles,203,6,0,1674.876847
1,520000,Ixelles,200,4,0,2600.000000
2,599000,Ixelles,160,4,0,3743.750000
3,599000,Ixelles,160,3,0,3743.750000
4,575000,Ixelles,171,3,0,3362.573099


In [199]:
df3.price_per_sqft.describe()

count    40395.000000
mean      2345.424291
std       1286.481438
min          4.166667
25%       1569.563490
50%       2180.616740
75%       2837.837838
max      67000.000000
Name: price_per_sqft, dtype: float64

In [202]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby("city_name"):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df], ignore_index = True)
    return df_out

df4 = remove_pps_outliers(df3)
df4.shape

(28838, 6)

In [204]:
df5 = df4.drop(["price_per_sqft"], axis=1)
df5.head()

,price,city_name,house_area,number_of_rooms,swimming_pool
0,550000,'s Gravenwezel,296,4,0
1,530000,'s Gravenwezel,180,3,0
2,595000,'s Gravenwezel,226,3,0
3,595000,'s Gravenwezel,225,3,0
4,575000,'s Gravenwezel,333,4,1


In [210]:
dummies = pd.get_dummies(df5.city_name)
dummies.head()

,'s Gravenwezel,Aalst,Aalter,Aartrijke,Aartselaar,Abolens,Achel,Adegem,Afsnee,Aische-En-Refail,...,Zepperen,Zingem,Zoerle-Parwijs,Zomergem,Zonhoven,Zulte,Zutendaal,Zwevegem,Zwijnaarde,other
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
df6 = pd.concat([df5,dummies.drop("other", axis=1)], axis=1)
df6.head()

,price,city_name,house_area,number_of_rooms,swimming_pool,'s Gravenwezel,Aalst,Aalter,Aartrijke,Aartselaar,...,Zelzate,Zepperen,Zingem,Zoerle-Parwijs,Zomergem,Zonhoven,Zulte,Zutendaal,Zwevegem,Zwijnaarde
0,550000,'s Gravenwezel,296,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,530000,'s Gravenwezel,180,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,595000,'s Gravenwezel,226,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,595000,'s Gravenwezel,225,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,575000,'s Gravenwezel,333,4,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
df7 = df6.drop("city_name", axis=1)
df7.head()

,price,house_area,number_of_rooms,swimming_pool,'s Gravenwezel,Aalst,Aalter,Aartrijke,Aartselaar,Abolens,...,Zelzate,Zepperen,Zingem,Zoerle-Parwijs,Zomergem,Zonhoven,Zulte,Zutendaal,Zwevegem,Zwijnaarde
0,550000,296,4,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,530000,180,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,595000,226,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,595000,225,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,575000,333,4,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
df7.shape

(28838, 675)

In [215]:
X = df7.drop("price", axis=1)
X.head()

,house_area,number_of_rooms,swimming_pool,'s Gravenwezel,Aalst,Aalter,Aartrijke,Aartselaar,Abolens,Achel,...,Zelzate,Zepperen,Zingem,Zoerle-Parwijs,Zomergem,Zonhoven,Zulte,Zutendaal,Zwevegem,Zwijnaarde
0,296,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,180,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,226,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,225,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,333,4,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
y = df7.price
y.head()

0    550000
1    530000
2    595000
3    595000
4    575000
Name: price, dtype: int64

In [217]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [218]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.7611986834715978

In [219]:
X.columns

Index(['house_area', 'number_of_rooms', 'swimming_pool', ''s Gravenwezel',
       'Aalst', 'Aalter', 'Aartrijke', 'Aartselaar', 'Abolens', 'Achel',
       ...
       'Zelzate', 'Zepperen', 'Zingem', 'Zoerle-Parwijs', 'Zomergem',
       'Zonhoven', 'Zulte', 'Zutendaal', 'Zwevegem', 'Zwijnaarde'],
      dtype='object', length=674)

In [ ]:
def predict_price(city_name, house_area, swimming_pool, number_of_rooms):
    city_name_index = np.where(X.columns=city_name)[0][0]
    
    x = np.zeros(len(X.columns))
    x[0] = house_area
    x[1] = swimming_pool
    x[2] = number_of_rooms
    if city_name_index >= 0:
        x[city_name_index] = 1
    
    return lr_clf.predict([x])[0]

In [220]:
import pickle
with open("immo_machine_learning.pickle", "wb") as f:
    pickle.dump(lr_clf, f)

In [221]:
import json
columns = {
    "data_columns" : [col.lower() for col in X.columns]
}

with open ("columns.json", "w") as f:
    f.write(json.dumps(columns))

In [76]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
for col in data.columns:
    print(col)

Unnamed: 0
postal_code
city_name
type_of_property
price
number_of_rooms
house_area
fully_equipped_kitchen
open_fire
terrace
garden
surface_of_the_land
number_of_facades
swimming_pool
state_of_the_building
lattitude
longitude
province
region


In [77]:
data = data.drop(["Unnamed: 0"], axis=1)

In [78]:
data.head()

,postal_code,city_name,type_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,1050,Ixelles,0,340000,6,203,1,0,1,0,95,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
1,1050,Ixelles,0,520000,4,200,0,0,0,0,69,2,0,to renovate,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
2,1050,Ixelles,0,599000,4,160,1,0,1,1,100,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
3,1050,Ixelles,0,599000,3,160,1,0,1,1,130,2,0,good,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
4,1050,Ixelles,0,575000,3,171,0,0,0,0,46,2,0,just renovated,4.381571,50.822285,Bruxelles-Capitale,Bruxelles


In [ ]:
df2 = df1(["price", "city_name", "house_area", "number_of_rooms", "swimming_pool"]

In [79]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data[["city_name", 
     "state_of_the_building",
     "province", 
     "region"]] = data[["city_name", "state_of_the_building", "province", "region"]].apply(le.fit_transform)

data.head()

,postal_code,city_name,type_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,1050,497,0,340000,6,203,1,0,1,0,95,2,0,3,4.381571,50.822285,3,0
1,1050,497,0,520000,4,200,0,0,0,0,69,2,0,4,4.381571,50.822285,3,0
2,1050,497,0,599000,4,160,1,0,1,1,100,2,0,3,4.381571,50.822285,3,0
3,1050,497,0,599000,3,160,1,0,1,1,130,2,0,1,4.381571,50.822285,3,0
4,1050,497,0,575000,3,171,0,0,0,0,46,2,0,2,4.381571,50.822285,3,0


In [162]:
labels = data["price"]
train1 = data.drop(["price"], axis=1)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train1, labels, test_size=0.1)
reg.fit(x_train, y_train)

LinearRegression()

In [163]:
reg.score(x_test,y_test)

0.46510234589783195

In [61]:
train1.head()

,city_name,type_of_property,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,497,0,6,203,1,0,1,0,95,2,0,3,4.381571,50.822285,3,0
1,497,0,4,200,0,0,0,0,69,2,0,4,4.381571,50.822285,3,0
2,497,0,4,160,1,0,1,1,100,2,0,3,4.381571,50.822285,3,0
3,497,0,3,160,1,0,1,1,130,2,0,1,4.381571,50.822285,3,0
4,497,0,3,171,0,0,0,0,46,2,0,2,4.381571,50.822285,3,0


In [45]:
data.corrwith(data["price"], axis=0, method='pearson')

postal_code              -0.131703
city_name                 0.057371
type_of_property         -0.118791
price                     1.000000
number_of_rooms           0.418513
house_area                0.529055
fully_equipped_kitchen    0.026582
open_fire                 0.178116
terrace                   0.126296
garden                    0.113077
surface_of_the_land       0.148578
number_of_facades         0.133420
swimming_pool             0.170617
state_of_the_building    -0.098732
lattitude                -0.078736
longitude                 0.208369
province                 -0.242668
region                   -0.245291
dtype: float64

In [ ]:
train1 = data.drop(["price"], axis=1)

In [46]:
import statsmodels.api as sm

In [47]:
train2 = sm.add_constant(train1)
model = sm.OLS(labels, train2).fit()
model.pvalues

const                      3.380212e-97
postal_code                5.219075e-75
city_name                  1.633629e-08
type_of_property          6.137007e-179
number_of_rooms            0.000000e+00
house_area                 0.000000e+00
fully_equipped_kitchen     1.354695e-01
open_fire                  7.489270e-75
terrace                   2.063969e-127
garden                     5.273273e-21
surface_of_the_land        1.002771e-85
number_of_facades          6.737034e-46
swimming_pool             2.604819e-106
state_of_the_building      2.659392e-76
lattitude                  3.475885e-72
longitude                 3.459856e-114
province                   4.917187e-01
region                    3.179977e-211
dtype: float64

In [49]:
#Backward Elimnation
cols = list(train1.columns)
pmax = 1
while (len(cols)>0):
    p = []
    train2 = train1[cols]
    train2 = sm.add_constant(train2)
    model = sm.OLS(labels, train2).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break

selected_features_BE = cols
print(selected_features_BE)

['postal_code', 'city_name', 'type_of_property', 'number_of_rooms', 'house_area', 'open_fire', 'terrace', 'garden', 'surface_of_the_land', 'number_of_facades', 'swimming_pool', 'state_of_the_building', 'lattitude', 'longitude', 'region']


In [51]:
train1.head()

,postal_code,city_name,type_of_property,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,1050,497,0,6,203,1,0,1,0,95,2,0,3,4.381571,50.822285,3,0
1,1050,497,0,4,200,0,0,0,0,69,2,0,4,4.381571,50.822285,3,0
2,1050,497,0,4,160,1,0,1,1,100,2,0,3,4.381571,50.822285,3,0
3,1050,497,0,3,160,1,0,1,1,130,2,0,1,4.381571,50.822285,3,0
4,1050,497,0,3,171,0,0,0,0,46,2,0,2,4.381571,50.822285,3,0
